In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA

In [2]:
dataset=pd.read_csv("nvidia_stock_data.csv")
dataset.columns

Index(['Unnamed: 0', 'Datetime', 'Open', 'High', 'Low', 'Close', 'Volume',
       'Dividends', 'Stock Splits'],
      dtype='object')

In [3]:
dataset.tail(4)

,Unnamed: 0,Datetime,Open,High,Low,Close,Volume,Dividends,Stock Splits
773,773,2025-10-10 14:15:00-04:00,187.630005,188.287399,187.270004,187.720001,3872425,0.0,0.0
774,774,2025-10-10 14:30:00-04:00,187.720001,187.960007,187.229996,187.240005,2754511,0.0,0.0
775,775,2025-10-10 14:45:00-04:00,187.250000,187.585007,186.619995,186.990005,4050620,0.0,0.0
776,776,2025-10-10 15:00:00-04:00,186.960007,187.550003,186.850006,186.860001,4189866,0.0,0.0


In [35]:
N = len(dataset)
train_len = N - 20
test_len = 20

train = dataset[:train_len]
test = dataset[train_len:train_len + test_len]


In [37]:
model=ARIMA(train['Open'],order=(1,0,1))
model=model.fit()
print(model.summary())

                               SARIMAX Results                                
Dep. Variable:                   Open   No. Observations:                  757
Model:                 ARIMA(1, 0, 1)   Log Likelihood                -804.241
Date:                Fri, 10 Oct 2025   AIC                           1616.481
Time:                        21:29:02   BIC                           1634.999
Sample:                             0   HQIC                          1623.613
                                - 757                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        181.4413      6.883     26.362      0.000     167.952     194.931
ar.L1          0.9957      0.004    278.413      0.000       0.989       1.003
ma.L1          0.0749      0.025      2.979      0.0

In [38]:
model.params

const     181.441271
ar.L1       0.995719
ma.L1       0.074924
sigma2      0.486962
dtype: float64

In [39]:
model.pvalues

const     3.731028e-153
ar.L1      0.000000e+00
ma.L1      2.893110e-03
sigma2     0.000000e+00
dtype: float64

In [60]:
forecasts=model.forecast(45)
forecasts

757    194.153218
758    194.098795
759    194.044605
760    193.990647
761    193.936920
762    193.883423
763    193.830155
764    193.777115
765    193.724302
766    193.671715
767    193.619353
768    193.567216
769    193.515302
770    193.463610
771    193.412139
772    193.360889
773    193.309858
774    193.259045
775    193.208450
776    193.158072
777    193.107909
778    193.057962
779    193.008228
780    192.958706
781    192.909397
782    192.860299
783    192.811412
784    192.762733
785    192.714263
786    192.666001
787    192.617945
788    192.570094
789    192.522449
790    192.475008
791    192.427770
792    192.380734
793    192.333899
794    192.287265
795    192.240830
796    192.194595
797    192.148557
798    192.102716
799    192.057072
800    192.011623
801    191.966369
Name: predicted_mean, dtype: float64

In [62]:
last_time

Timestamp('2025-10-10 15:00:00-0400', tz='UTC-04:00')

In [71]:
from datetime import datetime

n_forecasts = len(forecasts[20:])

last_time=pd.to_datetime(test['Datetime'].iloc[-1])

future_times = [
    last_time + pd.Timedelta(minutes=5 * (i + 1))
    for i in range(n_forecasts)
]

# the next forecasts:
forecast_df = pd.DataFrame({
    "Datetime": future_times,
    "Open": forecasts[20:]
})

forecast_df

,Datetime,Open
777,2025-10-10 15:05:00-04:00,193.107909
778,2025-10-10 15:10:00-04:00,193.057962
779,2025-10-10 15:15:00-04:00,193.008228
780,2025-10-10 15:20:00-04:00,192.958706
781,2025-10-10 15:25:00-04:00,192.909397
782,2025-10-10 15:30:00-04:00,192.860299
783,2025-10-10 15:35:00-04:00,192.811412
784,2025-10-10 15:40:00-04:00,192.762733
785,2025-10-10 15:45:00-04:00,192.714263
786,2025-10-10 15:50:00-04:00,192.666001


In [72]:
len(forecast_df)

25

In [73]:
forecast_df.to_csv("forecasts.csv")

In [67]:
train[["Datetime","Open"]].to_csv("train.csv")

In [68]:
test[["Datetime","Open"]].to_csv("test.csv")

In [70]:
import joblib

joblib.dump(model,"model_arima.pkl")

['model_arima.pkl']